In [101]:
import multiprocessing as mp
import time
import numpy as np
import math

"""this method creates a list of chunks which will be used for dividing the matrix among processors"""
def div(rows, n):
    div_list = []
    num = n
    while num > 0:
        # if the number of rows is larger than the number of processors then divide
        if rows > num:
            x = math.ceil(rows / num)
            div_list.append(int(x))
            rows -= x
            num -= 1
        elif rows == 0:
            div_list.append(0)
            num -= 1
        else:
            div_list.append(1)
            rows -= 1
            num -= 1
    return div_list


"""Sequential matrix multiplication"""
def multiply_matrices(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
      for j in range(B.shape[1]):
        for k in range(B.shape[0]):
          # write your code here
          C[i,j] += A[i,k] * B[k,j]
    return C




""" Parallel implementation """
def mat_mult_parallel(A, B, index):

  r = B.shape[1]
  for i in range(A.shape[0]):
      for j in range(B.shape[1]):
          sharedMemArr[index*r + i*r + j] = np.sum(A[i,:] * B[:,j])

""" Function for running parallel computation of matrix multiplication"""
def run_parallel(A, B, numProcessors, sharedMemArr):
    processes = []
    m = 0
    n = A.shape[0]

    for i in range(numProcessors):
      x = div(A.shape[0], numProcessors)
      A_slice = A[m:m + x[i]]
      p = multiprocessing.Process(target=mat_mult_parallel, args=(A_slice, B,m ))
      p.start()
      m += x[i]
      processes.append(p)
    if 1 in x:
        return multiply_matrices_vectorized(A, B)
    for p in processes:
        p.join()

    C = np.reshape(sharedMemArr, (n,B.shape[1]))	
    return C


"""Vectorized implementation of matrix multiplication. 
Used in the parallel approach to help with smaller matrices"""
def multiply_matrices_vectorized(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    # implement here your vectorized solution
    for i in range(A.shape[0]):
      for j in range(B.shape[1]):
        C[i,j] += np.sum(A[i,:] * B[:,j])
    return C


def run_sequential(A, B):
  matOut = multiply_matrices(A, B)
  return matOut


numProcessors = mp.cpu_count()

print("""
          Welcome! Please put your input according to the prompt
          """)
print("You have", numProcessors, "cores in your PC.","\n")
print("-" *100)
print("Using default 4 cores as of now")
print("-" *100)

numProcessors = 4
rows1 = int(input("How many rows would you want in your first matrix?: "))
cols1 = int(input("How many columns would you want in your first matrices?: "))
print("You put", rows1, "rows", cols1,"columns")
rows2 = int(input("How many rows would you want in your second matrix?: "))
cols2 = int(input("How many columns would you want in second matrices?: "))
print("You put", rows1, "rows", cols1,"columns")

if cols1 != rows2:
    print("Your matrices cannot do matrix multiplication")
else:

    print("Awesome! Doing matrix multiplication!")
    A = np.random.randint(low = 10,high = 500, size = (rows1,cols1))
    B = np.random.randint(low = 10,high = 500, size = (rows2,cols2))

    sharedMemArr = mp.Array('i', rows1*cols2) 

    time_start = time.time()

    C = run_parallel(A, B, numProcessors, sharedMemArr)
    time_end = time.time()
    parallel_time = time_end - time_start
    print('Time function multiply_matrices in parallel: %.6f seconds'% (parallel_time))

    time_start = time.time()
    C_ = multiply_matrices(A, B)
    time_end = time.time()
    sequential_time = time_end - time_start

    print('Time function multiply_matrices: %.6f seconds'% (sequential_time))
    mul = np.matmul(A, B)
    """Here for checking if the answer is correct"""
    # print("Matrix out:",C == mul)

    print("\n"*2 + "Here the parallel algorithm runs {1} times faster than the sequential approach".format(numProcessors, round(sequential_time/parallel_time,3)))




          Welcome! Please put your input according to the prompt
          
You have 2 cores in your PC. 

----------------------------------------------------------------------------------------------------
Using default 4 cores as of now
----------------------------------------------------------------------------------------------------
How many rows would you want in your first matrix?: 70
How many columns would you want in your first matrices?: 100
You put 70 rows 100 columns
How many rows would you want in your second matrix?: 100
How many columns would you want in second matrices?: 80
You put 70 rows 100 columns
Awesome! Doing matrix multiplication!
Time function multiply_matrices in parallel: 0.105871 seconds
Time function multiply_matrices: 0.511781 seconds


Here the parallel algorithm runs 4.834 times faster than the sequential approach
